In [1]:
import os
from backoff import on_exception, expo
import json
import requests
from collections import Counter
from datetime import datetime
from ratelimit import limits, sleep_and_retry, RateLimitException
import time
import urllib.request
import asyncio
import aiohttp
import nest_asyncio
from async_timeout import timeout
from requests.exceptions import HTTPError

In [2]:
data_path = "D:\\METIS\\data\\"
image_path = "D:\\METIS\\images\\"

images_saved = []
met_objects = []

In [3]:
url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects'
response = requests.get(url, allow_redirects=True)
all_objects = json.loads(response.text)
objectIDs = all_objects['objectIDs']

In [4]:
@sleep_and_retry
@on_exception(expo, RateLimitException, max_tries=8)
@limits(calls=300, period=300)
def get_object_details(object_id, img_url=True):
    url ='https://collectionapi.metmuseum.org/public/collection/v1/objects/{}'.format(object_id)
    response = requests.get(url, timeout=10)
    response_json = json.loads(response.text)
    met_objects.append(response_json)
    
    if img_url:
        return response_json['primaryImage']

@on_exception(expo, RateLimitException, max_tries=8)
@limits(calls=300, period=300)
def get_image(object_id, url):
    filename = image_path + "{}.jpg".format(object_id)
    im = urllib.request.urlretrieve(url, filename)
    images_saved.append({
                        'objectID': object_id, 
                        'url': url,
                        'filename': filename
                        })

In [5]:
img_url = get_object_details(1567)
get_image(1567, img_url)

In [6]:
images_saved

[{'objectID': 1567,
  'url': 'https://images.metmuseum.org/CRDImages/ad/original/79684.jpg',
  'filename': 'D:\\METIS\\images\\1567.jpg'}]

In [7]:
images_saved = []
met_objects = []

start = datetime.now()

for i, object_id in enumerate(objectIDs):
    if i % 1000 == 0:
        with open(data_path + "images_saved.json", "w") as write_file:
            json.dump(images_saved, write_file)   
            
        with open(data_path + "met_objects.json", "w") as write_file:
            json.dump(met_objects, write_file)  
            
        print("{}: {} elapsed".format(i, datetime.now() - start))
            
    try:
        img_url = get_object_details(object_id)
        get_image(object_id, img_url)
        
    except:
        pass
   # print(i, object_id)

In [8]:
start = datetime.now()

for i, object_id in enumerate(objectIDs[388000:400000]):
    if i % 1000 == 0:
        with open(data_path + "images_saved_388.json", "w") as write_file:
            json.dump(images_saved, write_file)   
            
        with open(data_path + "met_objects_388.json", "w") as write_file:
            json.dump(met_objects, write_file)  
            
        print("{}: {} elapsed".format(i + 388000, datetime.now() - start))
            
    try:
        img_url = get_object_details(object_id)
        get_image(object_id, img_url)
        
    except:
        pass

388000: 0:00:00.023002 elapsed
389000: 0:16:32.690189 elapsed
390000: 0:32:54.991917 elapsed
391000: 0:51:02.684042 elapsed
392000: 1:08:13.931526 elapsed
393000: 1:23:22.367842 elapsed
394000: 1:42:15.936664 elapsed
395000: 1:58:40.394503 elapsed
396000: 2:16:14.366381 elapsed
397000: 2:33:13.630194 elapsed
398000: 2:49:55.671094 elapsed
399000: 3:07:10.938912 elapsed


In [9]:
with open(data_path + "images_saved_388.json", "w") as write_file:
    json.dump(images_saved, write_file)   
            
with open(data_path + "met_objects_388.json", "w") as write_file:
    json.dump(met_objects, write_file)  

In [10]:
len(images_saved)

6097